GitHub repo for this: https://github.com/reuven/2024-09September-08-decorators

# Decorators

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

In [1]:
# let's assume that we have two simple functions, a and b, which return something very simple

def a():
    return f'A!\n'

def b():
    return f'B!\n'

In [2]:
print(a())
print(b())

A!

B!



In [3]:
# our company has mandated that we put dashed lines above and below everything we print
# one option is for us to rewrite the functions

lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [5]:
# the problem is that we will need to repeat this for every function we write!

# how can I rewrite this, such that I repeat myself as little as possible?
# option 1: write a function that takes our function as input, and returns the lines + our function's output

lines = '-' * 60 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'
    
print(with_lines(a))  # I'm not calling a
print(with_lines(b))  # I'm not calling b, either!

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [6]:
# the problem with the above is that we broke the existing/older API
# can we accomplish our goals, while keeping the old API?

# option 2: write a function that returns a function
# this is known as a closure -- we call a function, it returns a function, and then we assign that returned function to a variable

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
with_lines_b = with_lines(b)  
    
print(with_lines_a())  
print(with_lines_b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# How many functions?

1. We have the original `a` and `b` functions
2. We have `with_lines`, which we call, passing `a` and `b` to it
3. We have two functions, `with_lines_a` and `with_lines_b`, that we got back from calling `with_lines` with `a` and `b`

This is fine, but we can do even better -- we can assign the returned function from `with_lines(a)` not to `with_lines_a`, but rather to `a`. Similarly, we can assign the result of `with_lines(b)` back to `b`. As soon as we do that, we don't have direct access to the original `a` and `b` functions.

In [7]:
# option 3: don't define with_lines_a and with_lines_b. Rather, just assign back to a and b
# at this point, we have returned the original API, where we can call a() and b(), to working order

lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)   # once again, I'm not calling a

def b():
    return f'B!\n'
b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



In [8]:
# Finally, we can use Python's decorator syntax
# this is identical in functionality to what we just did and saw, but it looks a bit different


lines = '-' * 60 + '\n'

def with_lines(func):  
    def wrapper():    # closure
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines    # this line is equivalent to line 15
def a():
    return f'A!\n'
# a = with_lines(a)   

@with_lines   # this is equivalent to line 20
def b():
    return f'B!\n'
# b = with_lines(b)  
    
print(a())  
print(b()) 

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# Implications of this syntax

1. Don't have both line 12 + 15! There's normally no reason for line 15, but if you're teaching decorators, forgetting to comment out one of them can be not good.
2. If you want to apply the same decorator to a number of different functions, you must say @decorator above each definition.

# Can we decorate anything?

Yes, we can now decorate any function so long as it takes *zero* arguments.




In [14]:
lines = '-' * 60 + '\n'

def with_lines(func):    # this is called at function-definition time
    def wrapper(*args):  # this is called once for each time the decorated function is invoked
        print(locals())
        return f'{lines}{func(*args)}{lines}'  # here, we unroll *args into zero or more arguments
    return wrapper

@with_lines    # this line is equivalent to line 15
def a():
    return f'A!\n'

@with_lines   # this is equivalent to line 20
def b():
    return f'B!\n'

@with_lines
def addition(first, second):
    return f'{first} + {second} = {first+second}\n'

print(a())  
print(b()) 
print(addition(3, 5))

{'args': (), 'func': <function a at 0x11172a480>}
------------------------------------------------------------
A!
------------------------------------------------------------

{'args': (), 'func': <function b at 0x11172a3e0>}
------------------------------------------------------------
B!
------------------------------------------------------------

{'args': (3, 5), 'func': <function addition at 0x111728d60>}
------------------------------------------------------------
3 + 5 = 8
------------------------------------------------------------



# What happens to `addition`?

1. We define the `addition` function in lines 17-18, as per usual.
2. After its definition is complete, then Python runs `addition = with_lines(addition)`. It does this thanks to the `@with_lines` on line 16, which decorates the function on lines 17-18.
3. That runs `with_lines`, passing it `addition` as an argument.
4. The only thing that `with_lines` does is define `wrapper` and then return it. Note that `wrapper` is not invoked at this stage!
5. Whatever `with_lines` returns (`wrapper` in this case) is assigned back to `addition`, which we defined back in step 1, on lines 17-18.
6. When we invoke `addition`, we're really invoking `wrapper`, which grabs `func` from the local variables of its enclosing function, invokes the function, and then returns its value inside of a dashed string.

# Who needs this? Where do we use decorators?

1. If I have several functions that need to do the same thing, I can extract that functionality into a decorator, and simplify my code.
2. Timing of functions
3. Logging of functions
4. Security wrappers
5. Setting values before/after a function runs
6. Check inputs and/or filter them
7. Check outputs and/or filter them

Decorators allow us to hijack a function both when it is defined (then, the outer function is run) and when it is invoked (then, the inner function is run). These are fully fledged functions that we can use however we want.

Some examples in the Python world of where we use decorators:
- `@property`
- Pytest, many of its things use decorators
- Flask or Django
- FastAPI
- SQLAlchemy

# How do we write a decorator?

We're going to need an outer function and an inner function.

1. The outer function, the decorator, takes one argument, `func`. The argument is the function that will be decorated. The outer function is invoked *once*, just after the decorated function is defined.
2. The inner function, typically called `wrapper`, which takes `*args` (or `**kwargs`). This function is invoked once for each time the original (decorated) function is invoked. It can do whatever it wants with inputs and outputs.
3. After defining the inner function, the outer function returns `wrapper`.
4. Once that's done, we can decorate any function we want.

# Python tutor visualization of this decorator

https://pythontutor.com/render.html#code=lines%20%3D%20'-'%20*%2060%20%2B%20'%5Cn'%0A%0Adef%20with_lines%28func%29%3A%20%20%0A%20%20%20%20def%20wrapper%28*args%29%3A%20%20%20%20%23%20this%20function%20takes%20any%20number%20of%20positional%20arguments%0A%20%20%20%20%20%20%20%20return%20f'%7Blines%7D%7Bfunc%28*args%29%7D%7Blines%7D'%20%20%23%20here,%20we%20unroll%20*args%20into%20zero%20or%20more%20arguments%0A%20%20%20%20return%20wrapper%0A%0A%40with_lines%20%20%20%20%23%20this%20line%20is%20equivalent%20to%20line%2015%0Adef%20a%28%29%3A%0A%20%20%20%20return%20f'A!%5Cn'%0A%0A%40with_lines%20%20%20%23%20this%20is%20equivalent%20to%20line%2020%0Adef%20b%28%29%3A%0A%20%20%20%20return%20f'B!%5Cn'%0A%0A%40with_lines%0Adef%20addition%28first,%20second%29%3A%0A%20%20%20%20return%20f'%7Bfirst%7D%20%2B%20%7Bsecond%7D%20%3D%20%7Bfirst%2Bsecond%7D%5Cn'%0A%0Aprint%28a%28%29%29%20%20%0Aprint%28b%28%29%29%20%0Aprint%28addition%283,%205%29%29&cumulative=false&curInstr=47&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false

# Exercise: Shouter

1. Write a decorator, `shouter`, that decorates functions that return strings.
2. Any such function's output will return in `ALL CAPS` and with an explanation point at the end.
Example:

```python
@shouter
def hello(name):
    return f'Hello, {name}'

print(hello('Reuven'))   # output HELLO, REUVEN!   

In [18]:
def shouter(func):       # (3) shouter is invoked on hello after hello is defined
    def wrapper(*args):  # (4) wrapper is returned by shouter, and replaces the original hello
        return f'{func(*args)}!'.upper()
    return wrapper

@shouter           # (2) we decorate hello with shouter
def hello(name):   # (1) we define hello
    return f'Hello, {name}'

print(hello('Reuven'))   # (5) we call hello, but that really invokes wrapper, which then invokes the original hello

HELLO, REUVEN!


In [19]:
help(hello)

Help on function wrapper in module __main__:

wrapper(*args)



# Exercise: Timing of functions

Write a decorator, `timefunc`, that will not change the inputs or outputs of the decorated function *but* we will time how long it takes to run the function, and will record that to a file called `timing.txt`.

Every time I run the decorated function, I'll get one more line in `timing.txt` with the function name, when it started running, and how long it took to run. 

Hints:

- Normally, writing to a file with `w` will erase any previous contents. Instead, open the file with `a` ("append"), and then you'll write to the end.
- You can get a function's name from its `__name__` attribute.
- You can get the current Unix time (seconds since January 1st, 1970 at midnight) with `time.time()`.

I should be able to say the following:

```python
import random
import time

@timefunc
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))


In [20]:
print('hello')

hello


# This is a headline

- This is a bulleted list
- Two
- Three

This is a numbered list:
1. first
2. second
3. third

You can turn a cell into Markdown mode with `ESC` and `m`, because `ESC` puts it into command mode, and `m` makes it Markdown.

Similarly, `ESC` and `y` makes it a code (Python) cell.

In [24]:
import random
import time

def timefunc(func):        # outer function is run once per function definition
    def wrapper(*args):    # inner function is run once per function invocation
        start_time = time.time()
        value = func(*args)
        total_time = time.time() - start_time

        with open('timing.txt', 'a') as outfile:
            outfile.write(f'{func.__name__}\t{start_time:.03f}\t{total_time:.03f}\n')

        return value
    return wrapper

@timefunc
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))


5
20


In [25]:
!cat timing.txt

slow_add	1725816814.081	3.002
slow_mul	1725816817.086	0.000
slow_add	1725816838.658	1.003
slow_mul	1725816839.663	1.005


# Using the outer function

We've seen that the inner function, `wrapper`, has access to the local variables from the enclosing function. But we can do more than this -- we can use the outer function for *storage* that then even gets updated across our calls.



In [28]:
# I want to know how many times a function has been invoked, and print that counter to the screen every time
# we call the function

def count_calls(func):
    counter = 0     # local variable in the outer function
    def wrapper(*args):
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value
    return wrapper

@count_calls
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@count_calls
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))
print(slow_add(3, 4))
print(slow_mul(4, 9))
print(slow_add(6, 3))
print(slow_mul(4, 2))

UnboundLocalError: cannot access local variable 'counter' where it is not associated with a value

In [29]:
# we have to use nonlocal!

def count_calls(func):
    counter = 0     # local variable in the outer function
    
    def wrapper(*args):
        nonlocal counter    # this tells the compiler: counter is a local variable in the enclosing scope, *not* local 
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value
    return wrapper

@count_calls
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@count_calls
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))
print(slow_add(3, 4))
print(slow_mul(4, 9))
print(slow_add(6, 3))
print(slow_mul(4, 2))

slow_add: counter=1
5
slow_mul: counter=1
20
slow_add: counter=2
7
slow_mul: counter=2
36
slow_add: counter=3
9
slow_mul: counter=3
8


# Using `nonlocal`

We only need to use `nonlocal` when:

- We are assigning to a variable
- We want to affect a variable in the outer function's scope

If we are *modifying* or *mutating* an external value, then we don't need the `nonlocal`. What does this mean? If the nonlocal variable is a list or dict, then we can just say `mylist[3] = 'hello'`, without `nonlocal`. That's because only assignment to a variable is picked up by the Python compiler as affecting the local/global nature of variables.

# Memoization

Memoization is a caching technique that looks at the arguments to a function. If the arguments have been seen before, then we return the value from the previous call with those arguments. If the arguments are new, then we call the function for real, and then cache/store the return value for the next call.

This is for deterministic, simple functions that don't affect the system's state. 

```python
@memoize
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@memoize
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # really invoke slow_add here
print(slow_mul(4, 5))  # really invoke slow_mul here
print(slow_add(2, 3))  # return cached value for slow_add
print(slow_mul(2, 3))  # really invoke slow_mus
print(slow_add(2, 3))  # return cached value for slow_add
print(slow_mul(2, 3))  # return cached value for slow_mul
```

Write a decorator, `memoize`, that not only works, but prints whether it's really running the function or using the cache. (Hint: You can use a dict for the cache.)

In [31]:
def memoize(func):   # outer function, the decorator, called once
    cache = {}
    def wrapper(*args):
        if args not in cache:            # if this is the first time we're seeing these arguments...
            print(f'\tFirst time running {func.__name__} with {args}; calling for real')
            cache[args] = func(*args)    # ... call the function and store its return value in the cache
        else:
            print(f'\t\tRetrieving cached value {cache[args]} for {func.__name__} with args {args}')

        return cache[args]
    return wrapper


@memoize
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@memoize
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # really invoke slow_add here
print(slow_mul(4, 5))  # really invoke slow_mul here
print(slow_add(2, 3))  # return cached value for slow_add
print(slow_mul(2, 3))  # really invoke slow_mus
print(slow_add(2, 3))  # return cached value for slow_add
print(slow_mul(2, 3))  # return cached value for slow_mul


	First time running slow_add with (2, 3); calling for real
5
	First time running slow_mul with (4, 5); calling for real
20
		Retrieving cached value 5 for slow_add with args (2, 3)
5
	First time running slow_mul with (2, 3); calling for real
6
		Retrieving cached value 5 for slow_add with args (2, 3)
5
		Retrieving cached value 6 for slow_mul with args (2, 3)
6


# Exercise: `once_per_minute`

Write a decorator, `once_per_minute`, that only allows a decorated function to be run once per minute. Any more frequently than that, and it'll raise an exception. (`CalledTooSoonError`?)

```python
@once_per_minute
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@once_per_minute
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # works fine
print(slow_mul(4, 5))  # works fine
print(slow_add(2, 3))  # raises the exception here
print(slow_mul(2, 3))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 
```

In [33]:
class CalledTooSoonError(Exception):
    pass

def once_per_minute(func):
    last_run_at = 0

    def wrapper(*args):
        nonlocal last_run_at
        current_time = time.time()
        if current_time - last_run_at < 60:
            raise CalledTooSoonError('Too soon!')

        last_run_at = current_time
        
        return func(*args)
    return wrapper

@once_per_minute
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@once_per_minute
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # works fine
print(slow_mul(4, 5))  # works fine
print(slow_add(2, 3))  # raises the exception here
print(slow_mul(2, 3))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 


5
20


CalledTooSoonError: Too soon!

# Modifying function inputs and outputs

When we write a decorator, we have two opportunities to do something:

- When we define the function
- When we run the function

When we run the function, we get the inputs (in `*args`), and we return our outputs. This means that we can examine and filter the inputs if we want, or modify them. We can similiarly decide that only some of the outputs should be returned.



In [35]:
def mysum(*args):
    total = 0

    for one_number in args: 
        total += one_number

    return total

mysum(10, 20, 30)

60

In [36]:
mysum(10, 20, 'hello', 30)

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [38]:
# I can define an only_odds decorator 

def only_odds(func):
    def wrapper(*args):
        odd_args = []

        for one_arg in args:
            if one_arg % 2 == 1:  # if it's odd...
                odd_args.append(one_arg)

        return func(*odd_args)
    return wrapper


# now let's decorate the function!

@only_odds
def mysum(*args):
    total = 0

    for one_number in args: 
        total += one_number

    return total

mysum(10, 25, 30, 35)

60

In [39]:
# let's use a list comprehension!

# I can define an only_odds decorator 

def only_odds(func):
    def wrapper(*args):
        return func(*[one_arg
                      for one_arg in args
                      if one_arg % 2 == 1])
    return wrapper


# now let's decorate the function!

@only_odds
def mysum(*args):
    total = 0

    for one_number in args: 
        total += one_number

    return total

mysum(10, 25, 30, 35)

60

# Exercise: `only_ints`

Write a decorator that ignores the non-int arguments to a function. If we call `mysum` (the original) with some non-int arguments, the function will still work fine. You probably want to use `isinstance(OBJECT, int)` to check if something is an int.

In [42]:

def only_ints(func):
    def wrapper(*args):
        return func(*[one_arg
                      for one_arg in args
                      if isinstance(one_arg, int)])
    return wrapper


# now let's decorate the function!

@only_ints
def mysum(*args):
    total = 0

    for one_number in args: 
        total += one_number

    return total

mysum(10, 25, 30, 10.0, 35)

100

# How can a decorator take arguments?

1. What if we want to specify what type (or types) are acceptable for `only_ints`?
2. What if we want to specify how long we'll wait (not 60 seconds) for a function to run?
3. What if we want to specify the filename into which our logging info will go?

Let's think about how that might work. We know that 

    @decorator
    def myfunc():
        return 10

is rewritten to be

    def myfunc():
        return 10
    myfunc = decorator(myfunc)

We have three functions here -- the decorator, wrapper (which is returned), and the original function.

If we want the decorator to take arguments, we'll need to think about it differently:

    @decorator('a')
    def myfunc():
        return 10

is rewritten to be

    def myfunc():
        return 10
    myfunc = decorator('a')(myfunc)

This means that we don't call decorator(myfunc). Rather, we call decorator('a'), which returns a function, which is invoked on myfunc, which returns a function, which is assigned to myfunc.

In other words, we're going to need *three* nested functions:

- The outer function (the decorator) will now take the argument
- The middle function will take the decorated function
- Wrapper will remain the same.



In [45]:
# Let's rewrite only_ints to be only_of_type, and pass a type


def only_of_type(the_type):
    def middle(func):
        def wrapper(*args):
            return func(*[one_arg
                          for one_arg in args
                          if isinstance(one_arg, the_type)])
        return wrapper
    return middle


# now let's decorate the function!

@only_of_type(float)
def mysum(*args):
    total = 0

    for one_number in args: 
        total += one_number

    return total

mysum(10, 25, 30, 10.0, 35)

10.0

# Combining decorators

Can we stack decorators, using more than one on a function? Yes!

- When we define the function, the decorators are applied from the inside out. The decorator that's closest to the function we're decorating gets first dibs.
- When we invoke the function, the decorators are applied from the outside in.

Let's combine two of the decorators we defined.

In [46]:
@timefunc
@memoize
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
@memoize
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # 
print(slow_mul(4, 5))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 


	First time running slow_add with (2, 3); calling for real
5
	First time running slow_mul with (4, 5); calling for real
20
		Retrieving cached value 5 for slow_add with args (2, 3)
5
	First time running slow_mul with (2, 3); calling for real
6
		Retrieving cached value 5 for slow_add with args (2, 3)
5
		Retrieving cached value 6 for slow_mul with args (2, 3)
6


In [47]:
!cat timing.txt

slow_add	1725816814.081	3.002
slow_mul	1725816817.086	0.000
slow_add	1725816838.658	1.003
slow_mul	1725816839.663	1.005
wrapper	1725821324.774	1.004
wrapper	1725821325.781	0.000
wrapper	1725821325.781	0.000
wrapper	1725821325.782	0.000
wrapper	1725821325.782	0.000
wrapper	1725821325.782	0.000


How can we tell Python to use the real names of our functions, and not `wrapper`?

The answer is: A decorator. 

If we use `functools.wrap`, a decorator that we can apply to wrapper, it will grab the name and docstring, and make wrapper appear like the original function.

In [ ]:
import random
import time

def timefunc(func):        # outer function is run once per function definition
    def wrapper(*args):    # inner function is run once per function invocation
        start_time = time.time()
        value = func(*args)
        total_time = time.time() - start_time

        with open('timing.txt', 'a') as outfile:
            outfile.write(f'{func.__name__}\t{start_time:.03f}\t{total_time:.03f}\n')

        return value
    return wrapper

def memoize(func):   # outer function, the decorator, called once
    cache = {}
    def wrapper(*args):
        if args not in cache:            # if this is the first time we're seeing these arguments...
            print(f'\tFirst time running {func.__name__} with {args}; calling for real')
            cache[args] = func(*args)    # ... call the function and store its return value in the cache
        else:
            print(f'\t\tRetrieving cached value {cache[args]} for {func.__name__} with args {args}')

        return cache[args]
    return wrapper

@timefunc
@memoize
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
@memoize
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))  # 
print(slow_mul(4, 5))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 
print(slow_add(2, 3))  # 
print(slow_mul(2, 3))  # 
